<a href="https://colab.research.google.com/github/GemmaGorey/Dissertation/blob/main/notebooks/Dissertation_GG-benchmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initial Colab setup below - Run once only per session**


In [ ]:
# Environment Setup - run once per session then run all below

# Clone my github
!git clone https://github.com/GemmaGorey/Dissertation.git

# Install librabries
print("Installing required library versions...")
# Downgrade spaCy and NumPy to be compatible with PyTorch 2.2.2 and its dependencies as conficts with colab
!pip install "spacy<3.8" "numpy<2" -q

# Install PyTorch, torchvision, torchaudio, and librosa
!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu121 -q
!pip install librosa -q
print("Installation complete.")

# verify GPU and restart trigger
import torch
import os

if not torch.cuda.is_available():
    print("WARNING: GPU IS NOT AVAILABLE. Please go to Runtime > Change runtime type and select T4 GPU.")
else:
    print("\n GPU is available. Runtime will now restart to load correct library versions.")
    print(" After restarting, you can run the rest of your notebook cells.")
    # Colab runtime restart.
    os.kill(os.getpid(), 9)

Cloning into 'Dissertation'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 49 (delta 16), reused 5 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 15.52 KiB | 3.88 MiB/s, done.
Resolving deltas: 100% (16/16), done.
Installing required library versions...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 127.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 74.7 MB

**Below is the rest of the script**

Start once reconnected (tick on top RHS)


In [1]:
import time
import torch
import torch.nn as nn

print("--- CNN Training Loop Benchmark ---")

# --- 1. Setup Device ---
# This code automatically detects and uses the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Benchmark will use device: {device}")


# --- 2. Create Sample Data ---
# We'll create a dummy batch of data that looks like spectrograms.
# This simulates the input your model would receive.
BATCH_SIZE = 32
CHANNELS = 1  # Spectrograms are single-channel (like grayscale images)
N_MELS = 128  # Height of the spectrogram (number of Mel bands)
TIME_FRAMES = 431 # Width of the spectrogram (number of time frames)
NUM_CLASSES = 4 # Example: 4 emotion classes

# Create the dummy data and labels on the target device
dummy_spectrograms = torch.randn(BATCH_SIZE, CHANNELS, N_MELS, TIME_FRAMES).to(device)
dummy_labels = torch.randint(0, NUM_CLASSES, (BATCH_SIZE,)).to(device)

print(f"Simulated data shape: {dummy_spectrograms.shape}")


# --- 3. Define a Basic CNN Model ---
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        # The input features to the linear layer will depend on the pooling output size
        # For our input: 128x431 -> pool1 -> 64x215 -> pool2 -> 32x107
        self.fc1 = nn.Linear(32 * 32 * 107, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.fc1(x)
        return x

# Instantiate the model and move it to the GPU
model = SimpleCNN(num_classes=NUM_CLASSES).to(device)
print(f"\nModel loaded onto {device}.")


# --- 4. Setup Training Components ---
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# --- 5. Run and Time the Training Loop ---
print("Starting training loop for 10 epochs...")
start_time = time.time()

for epoch in range(10):
    # In a real scenario, you'd loop through batches of a dataset here.
    # For this benchmark, we'll just re-use the same dummy batch.

    # Forward pass
    outputs = model(dummy_spectrograms)
    loss = criterion(outputs, dummy_labels)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")

end_time = time.time()
total_duration = end_time - start_time
print("\n--- Benchmark Complete ---")
print(f"Total training time for 10 epochs on {device}: {total_duration:.2f} seconds")

--- CNN Training Loop Benchmark ---
Benchmark will use device: cuda
Simulated data shape: torch.Size([32, 1, 128, 431])

Model loaded onto cuda.
Starting training loop for 10 epochs...
Epoch [1/10], Loss: 1.4453
Epoch [2/10], Loss: 30.1388
Epoch [3/10], Loss: 8.8652
Epoch [4/10], Loss: 9.0162
Epoch [5/10], Loss: 13.1328
Epoch [6/10], Loss: 9.9128
Epoch [7/10], Loss: 8.0377
Epoch [8/10], Loss: 4.3783
Epoch [9/10], Loss: 2.3709
Epoch [10/10], Loss: 0.8175

--- Benchmark Complete ---
Total training time for 10 epochs on cuda: 1.20 seconds
